# 비디오(Video) 질의 응답 LLM (Gemini)

**주요 흐름**

1. `File API`를 사용하여 비디오 파일을 업로드합니다.
2. `GenerateContent` 요청을 통해 비디오에 대한 질문을 요청합니다.
3. 생성된 응답을 확인합니다.
4. 업로드한 Video 파일을 삭제합니다.


**중요:** 

- 본 튜토리얼의 `File API` 는 인증 및 접근을 위해 `API keys`를 사용합니다. 

- 업로드된 파일은 `API key`의 클라우드 프로젝트와 연결됩니다. 

다른 `Gemini API` 와 달리, `API key`는 `File API`에 업로드한 데이터에 대한 접근 권한도 부여하므로 `API key`를 안전하게 보관하는 데 특별히 주의해야 합니다.

**Reference**

- [Gemini API(Cookbook) - Video](https://ai.google.dev/api/rest/v1/models/generateContent#media)

**API KEY 발급**

- [링크](https://makersuite.google.com/app/apikey?hl=ko) 에서 API KEY를 발급받아주세요.
- 사용자의 Google API 키를 환경 변수 `GOOGLE_API_KEY`로 설정합니다.

`.env` 파일에 아래와 같이 입력합니다.

```
GOOGLE_API_KEY=<사용자의 API KEY>
```


In [ ]:
# LangSmith 추적 설정
# !pip install langchain-teddynote
from langchain_teddynote import logging
 
# 프로젝트 이름 입력
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


In [11]:
# LangSmith 추적 설정
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름 입력
logging.langsmith("CH04-Gemini-Video")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Gemini-Video


In [ ]:
# google pip install

!pip install google-generativeai

  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.2.0-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 71.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   ---------------------------------------- 14.0/14.0 MB 87.8 MB/s eta 0:00:00
Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl (9.3 kB)
Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
Using cached uritemplate-4.2.0-py3-none-any.whl (11 kB)

   -------------------- ------------------- 3/6 [google-api-python-client]
   -------------------- ------------------- 3/6 [google-api-python-client]
   -------------------- ------------------- 3/6 [google-api-python-client]
  Attempting uninstall: google-ai-generativelanguage
   -------------------- -------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.9 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 비디오 업로드

`Gemini API` 는 비디오 파일 형식을 직접 수용합니다. 

**제한사항**

- `File API`는 2GB 이하의 파일을 수용하며 프로젝트당 최대 20GB의 파일을 저장할 수 있습니다. 
- 파일은 2일 동안 유지되며 API에서 **다운로드할 수 없습니다**. 

본 예제는 테디노트 YouTube 채널에 게시된 비디오를 사용합니다. (다른 비디오로 교체하여 진행해도 좋습니다)

- [🧑‍💻 #PDF 전처리 할 수 있는 건 다 해봤음.](https://youtu.be/O3qFWRObAXw)

In [12]:
# 파일 다운로드 후 teddynote-sample-video.mp4 파일로 저장
!wget "https://www.dropbox.com/scl/fi/ugue14fyo010jgc7wuh4g/teddynote-sample-video.mp4?rlkey=wcsiktklt7jgoibsluft3m6z9&st=prv4p2uu&dl=1" -qO teddynote-sample-video.mp4


'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


아래 비디오 파일의 경로를 입력합니다.

In [13]:
# 비디오 파일 이름 지정
video_file_name = "teddynote-sample-video.mp4"

다음으로는 `File API`를 사용하여 비디오 파일을 업로드합니다.

In [20]:
# google ai studio 라이브러리를 사용

import google.generativeai as genai

# 파일 업로드 진행 메시지 출력
print(f"파일을 업로드 중입니다...")

# 파일 업로드 및 파일 객체 반환
video_file = genai.upload_file(path=video_file_name)

# 업로드 완료 메시지 및 파일 URI 출력
print(f"업로드 완료: {video_file.uri}")

파일을 업로드 중입니다...
업로드 완료: https://generativelanguage.googleapis.com/v1beta/files/s9d5np7u9mig


파일을 업로드한 후, `files.get` 을 호출하여 API가 파일을 성공적으로 완료되었는지 확인할 수 있습니다. 

`files.get`은  API 키가 속한 클라우드 프로젝트와 연관된 파일 API에 업로드된 파일을 확인할 수 있게 해줍니다. 

In [21]:
import time

# 비디오 파일 처리 상태 확인
while video_file.state.name == "PROCESSING":
    # 처리 완료 대기 메시지 출력
    print("비디오 업로드 및 전처리가 완료될 때까지 잠시만 기다려주세요...")
    # 10초 대기
    time.sleep(10)
    # 비디오 파일 상태 갱신
    video_file = genai.get_file(video_file.name)

# 처리 실패 시 예외 발생
if video_file.state.name == "FAILED":
    raise ValueError(video_file.state.name)

# 처리 완료 메시지 출력
print(
    f"\n비디오 처리가 완료되었습니다!\n이제 대화를 시작할 수 있어요: " + video_file.uri
)

비디오 업로드 및 전처리가 완료될 때까지 잠시만 기다려주세요...

비디오 처리가 완료되었습니다!
이제 대화를 시작할 수 있어요: https://generativelanguage.googleapis.com/v1beta/files/s9d5np7u9mig


비디오가 업로드된 후, `generate_content` 함수를 사용하여 Video 에 대한 질문을 요청할 수 있습니다.

In [23]:
# 프롬프트
prompt = "이 영상에 대해서 짧게 요약해 줄 수 있나요?"

# 모델을 Gemini 2.5 Flash로 설정
model = genai.GenerativeModel(model_name="models/gemini-2.5-flash")

# LLM 답변 요청
response = model.generate_content(
    [prompt, video_file], request_options={"timeout": 600}
)
# 결과 출력
print(response.text)

이 영상은 **LangGraph를 활용한 고급 문서 전처리 기법**에 대해 설명합니다. PDF와 같은 문서에서 텍스트뿐만 아니라 차트, 이미지, 표 등의 비정형 데이터를 효율적으로 추출하고, 이를 기반으로 **검색 증강 생성(RAG) 시스템의 답변 품질을 높이는 방법**을 다룹니다.

주요 내용은 다음과 같습니다:

1.  **문제점 인식**: 일반적인 PDF 파싱으로는 문서 내 표나 이미지가 누락되거나 단순 텍스트로만 추출되어 질문에 대한 맥락적 답변에 한계가 있습니다. 높은 품질의 답변을 위해서는 문서 전처리가 매우 중요합니다.
2.  **레이아웃 파싱 활용**: Upstage AI의 레이아웃 파싱 알고리즘을 사용하여 PDF 문서 내의 텍스트, 이미지, 표와 같은 각 요소를 바운딩 박스 좌표와 함께 개별 요소로 추출합니다.
3.  **요소별 심층 처리**:
    *   **이미지**: 추출된 이미지의 좌표를 이용하여 PDF에서 실제 이미지를 잘라내어 파일로 저장하고, 이미지의 제목 및 요약본을 텍스트 형태로 생성하여 이미지의 의미를 부여합니다.
    *   **표**: 표를 HTML 형태로 추출하고, 표 내용을 요약하거나 핵심 개체를 추출하여 벡터 검색에 활용하기 용이하게 만듭니다. 또한, 복잡한 표 레이아웃을 Markdown 형식으로 깔끔하게 변환하여 가독성을 높입니다.
    *   **텍스트**: 페이지별로 텍스트 요약본을 생성하여 검색 시 요약 정보와 원문을 함께 활용할 수 있도록 합니다.
4.  **데이터 구조화**: 모든 처리된 데이터를 `GraphState`라는 통합된 딕셔너리 객체에 파일 경로, 페이지 메타데이터, 분리된 요소, 요약본 등 다양한 형태로 저장하여 추후 RAG 시스템에서 쉽게 접근하고 활용할 수 있도록 합니다.
5.  **성능 최적화**: PDF를 페이지 단위로 분할하여 처리하고, 텍스트 요약 등 반복적인 작업을 배치(Batch) 처리하여 속도를 크게 개선합니다.

결론적으로, 이 영상은 단순한 텍스트 추출을 넘어 문서의 시

아래는 스트림 출력 예제입니다. (`stream=True` 옵션 추가)


In [24]:
# 프롬프트 생성
prompt = "이 영상에서 Gencon 관련 언급한 부분의 시간을 알려주고, 어떤 내용을 말했는지 알려주세요."

# 모델을 Gemini 2.5 Flash로 설정
model = genai.GenerativeModel(model_name="models/gemini-2.5-flash")

# LLM 스트림 답변 요청
response = model.generate_content(
    [prompt, video_file], request_options={"timeout": 600}, stream=True
)

# 생성된 콘텐츠 출력
for chunk in response:
    print(chunk.text, end="", flush=True)

네, 영상에서 Gencon 관련 언급한 부분은 **00:26초부터 00:44초까지**입니다.

**주요 내용은 다음과 같습니다:**

*   발표자는 **"Gencon 2024 AI Conference for DEV"**라는 이벤트를 언급했습니다.
*   이 행사는 **9월 20일 (금요일) 서울 코엑스**에서 개최됩니다.
*   라이브 방송 중 퀴즈를 통해 **무료 참가 기회를 3명에게 제공**하며, 이 퀴즈의 **암호는 라이브 방송 중에 알려줄 것**이라고 말했습니다.
*   라이브 방송을 듣는 분들은 해당 암호를 설문에 작성하여 참여할 수 있다고 안내했습니다.
*   9월 20일에 시간이 되는 분들은 참석하면 도움이 될 것이라고 권유했습니다.

## 파일 삭제

파일은 2일 후 자동으로 삭제되거나 `files.delete()`를 사용하여 수동으로 삭제할 수 있습니다.

In [25]:
# 파일 삭제
genai.delete_file(video_file.name)

# 삭제 완료 메시지 출력
print(f"영상을 삭제했습니다: {video_file.uri}")

영상을 삭제했습니다: https://generativelanguage.googleapis.com/v1beta/files/s9d5np7u9mig
